# Reconnaissance des Digits basée sur les enregistrements Audios 

### Importer les bibliothéques necessaires 

In [1]:
from Tools.tools import rec
from Tools.tools import collection
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_style("whitegrid", {'axes.grid' : False})
sb.set(font_scale = 2)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier 

### Collection 

In [2]:
# collection()

#### 1. Importer votre DataSet

In [3]:
mydata = pd.read_csv('./DataSet/'+os.listdir('./DataSet/')[0])
mydata.shape

(230, 13)

##### Print

In [4]:
mydata.head()

,Fe1,Fe2,Fe3,Fe4,Fe5,Fe6,Fe7,Fe8,Fe9,Fe10,Fe11,Fe12,Target
0,13.925822,-9.692369,3.753776,1.327168,2.975534,-7.993735,-0.575305,-7.071838,-8.511768,-1.650743,-2.721607,-5.412130,0.0
1,14.055519,-8.598112,-0.383220,-7.007748,-1.833112,-7.642512,4.104115,-0.713202,-4.826460,-2.594968,-5.888919,-5.624817,1.0
2,13.864094,-8.834997,1.500129,-7.639205,2.511519,-0.771141,4.451372,-5.286776,-12.323968,-8.869542,-5.942759,-6.046688,2.0
3,14.147140,-8.770026,2.766291,-7.528646,-3.671171,-10.579631,1.028021,-1.223803,-4.780219,-0.336830,0.487315,-3.639368,3.0
4,12.662124,-21.488978,-4.017699,-10.286525,-3.537687,-6.191114,2.452089,-0.002252,-2.034508,-1.515711,0.145741,0.618533,4.0


#### 2. Checker les variables quantitatives/qualitatives et les valeurs manquantes 

In [5]:
#mydata.describe()
#mydata.corr().style.background_gradient(cmap='coolwarm').set_precision(2)
mydata.isna().any()

Fe1       False
Fe2       False
Fe3       False
Fe4       False
Fe5       False
Fe6       False
Fe7       False
Fe8       False
Fe9       False
Fe10      False
Fe11      False
Fe12      False
Target    False
dtype: bool

#### 3. Visualiser les targets

In [6]:
mydata['Target']

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
225    5.0
226    6.0
227    7.0
228    8.0
229    9.0
Name: Target, Length: 230, dtype: float64

#### 4. Notre variable target (Y) est 'gender', Récuprer X et y à partir du jeu de données 

In [7]:
y = mydata['Target']
X = mydata.iloc[:,:-1]
X.shape

(230, 12)

In [8]:
y.shape

(230,)

#### 5. Diviser la DataSet en donneés d'apprentissage et de test (20% pour le test)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=20/100)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(184, 12) (46, 12)
(184,) (46,)


#### 6. Appliquer une normalisation centrée-réduite aux données en utilisant "StandardScaler"

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X.copy())
x_train_scaled = scaler.transform(x_train)

#And we want to apply this transformation to a new dataset called New_data
x_test_scaled = scaler.transform(x_test)

In [11]:
print(x_train_scaled.mean(), x_train_scaled.std())
print(x_test_scaled.mean(), x_test_scaled.std())

0.0015639963631606199 1.0054202941614272
-0.006255985452642043 0.9779934594162264


#### 7. Développer votre meilleur modèle de classification

##### SVM

In [19]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
#best_C = np.logspace(1,9,num=9,base=10,dtype='int')  # range() avec pas logarithmique (de 2^1 à 2^9)
#best_C = range(1, 100, 10)
best_C = range(1, 10)
best_C = [i/100000 for i in range(18830, 18840)]
# 138 : 0.6518018018018019
# 100 : 0.6518018018018019
#   1 : 0.6783783783783783
#   2 : 0.6840840840840843
# .19 : 0.6837837837837838
#.1883
print(best_C)
grid = GridSearchCV(LinearSVC(), {'C': best_C, 'dual': [False]})
grid.fit(x_train_scaled, y_train)
print("Best score : ", grid.best_score_)
print("Best param : ", grid.best_params_)

[0.1883, 0.18831, 0.18832, 0.18833, 0.18834, 0.18835, 0.18836, 0.18837, 0.18838, 0.18839]
Best score :  0.679129129129129
Best param :  {'C': 0.1883, 'dual': False}


In [20]:
model = grid.best_estimator_
model.fit(x_train_scaled, y_train)

LinearSVC(C=0.1883, dual=False)

##### Tensorflow

In [24]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train_scaled.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement du modèle
history = model.fit(x_train_scaled,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
2/2 [==============================] - 0s 4ms/step - loss: 2.3268 - accuracy: 0.0761
Epoch 2/20
2/2 [==============================] - 0s 8ms/step - loss: 2.2302 - accuracy: 0.2065
Epoch 3/20
2/2 [==============================] - 0s 5ms/step - loss: 2.1599 - accuracy: 0.3859
Epoch 4/20
2/2 [==============================] - 0s 7ms/step - loss: 2.0949 - accuracy: 0.4130
Epoch 5/20
2/2 [==============================] - 0s 8ms/step - loss: 2.0281 - accuracy: 0.4728
Epoch 6/20
2/2 [==============================] - 0s 8ms/step - loss: 1.9618 - accuracy: 0.5054
Epoch 7/20
2/2 [==============================] - 0s 7ms/step - loss: 1.8907 - accuracy: 0.5217
Epoch 8/20
2/2 [==============================] - 0s 6ms/step - loss: 1.8178 - accuracy: 0.5217
Epoch 9/20
2/2 [==============================] - 0s 5ms/step - loss: 1.7381 - accuracy: 0.5435
Epoch 10/20
2/2 [==============================] - 0s 6ms/step - loss: 1.6555 - accuracy: 0.5543
Epoch 11/20
2/2 [======================

In [25]:
# évaluation du modèle
test_loss, test_acc = model.evaluate(x_test_scaled, y_test)
print('test_acc: ',test_acc)

# Predictions
predictions = model.predict(x_test_scaled)
print("Meilleure predictions pour : ", np.argmax(predictions[0]))

2/2 [==============================] - 0s 3ms/step - loss: 1.2662 - accuracy: 0.6522
test_acc:  0.6521739363670349
Meilleure predictions pour :  2


##### XGBoost

In [58]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(x_train)
dtest = xgb.DMatrix(x_test)
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)


XGBoostError: [16:46:53] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:58: Check failed: preds.Size() == info.labels_.Size() (184 vs. 0) :  labels are not correctly providedpreds.size=184, label.size=0, Loss: binary:logistic

In [72]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Optimiser la structure des données
data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(x_train,y_train)

preds = xg_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

# k-fold Cross Validation
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
print(cv_results.head())

print("Meilleur : ", (cv_results["test-rmse-mean"]).tail(1))

RMSE: 2.338177
   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0         4.748199        0.113509        4.776503       0.275267
1         4.414920        0.090305        4.481198       0.268509
2         4.141613        0.081497        4.225046       0.257551
3         3.886535        0.082728        3.991379       0.229127
4         3.646764        0.088456        3.779880       0.223214
Meilleur :  49    2.299618
Name: test-rmse-mean, dtype: float64


# Application en Temps Réel

In [26]:
# del rec
from Tools.tools import rec

In [46]:
pred = rec(scaler, model)

Attention, l'enregistrement commence dans :
5
4
3
2
1
0
Prononcer votre Digit : 
------------------
Digit (prédiction) :  [0.00619283 0.2732236  0.00277539 0.08298996 0.01651725 0.4003243
 0.02103643 0.00259798 0.00467279 0.18966942]
------------------


In [47]:
# Pour Tensorflow :

print("Prediction : ", np.argmax(pred[0]))
for p in pred[0]:
    print(p)

Prediction :  5
0.006192827
0.2732236
0.0027753888
0.08298996
0.016517254
0.4003243
0.021036427
0.002597982
0.0046727904
0.18966942
